# Datadog LLMObs Trace Extractor

Queries the **Datadog LLM Observability Export API** — a dedicated endpoint for reading LLMObs spans programmatically.

**Reference:** [docs.datadoghq.com/llm_observability/evaluations/export_api](https://docs.datadoghq.com/llm_observability/evaluations/export_api)

| | Value |
|---|---|
| **API endpoint** | `POST https://api.us3.datadoghq.com/api/v2/llm-obs/v1/spans/events/search` |
| **Content-Type** | `application/vnd.api+json` |
| **Time range** | 21 Feb 2026, 16:40–18:30 Bangkok (UTC+7) |
| **span_kind** | `workflow` |
| **ml_app** | `gemini-ss6_1` |
| **Output** | `./datasets/ss6_1_llmobs_traces.jsonl` |

## 1. Install Dependencies

In [25]:
!pip install -q requests python-dotenv pandas

## 2. Setup & Configuration

In [26]:
import json
import os
import requests
from datetime import datetime, timezone, timedelta
from pathlib import Path

import pandas as pd
from dotenv import load_dotenv

load_dotenv(override=True)

DD_API_KEY = os.environ["DD_API_KEY"]
DD_APP_KEY = os.environ["DD_APP_KEY"]
DD_SITE    = os.getenv("DD_SITE", "us3.datadoghq.com")

print(f"✅ Datadog site : {DD_SITE}")
print(f"   API key     : {DD_API_KEY[:8]}...")
print(f"   App key     : {DD_APP_KEY[:8]}...")

✅ Datadog site : us3.datadoghq.com
   API key     : e1845489...
   App key     : caec2402...


## 3. Define Query Parameters

Time range is **Bangkok (UTC+7)** — converted to UTC for the API.

In [27]:
BKK = timezone(timedelta(hours=7))

FROM_DT = datetime(2026, 2, 21, 16, 40, 0, tzinfo=BKK)
TO_DT   = datetime(2026, 2, 21, 18, 40, 0, tzinfo=BKK)

FROM_ISO = FROM_DT.astimezone(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")
TO_ISO   = TO_DT.astimezone(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")

SPAN_KIND = "workflow"
ML_APP    = "gemini-ss6_1"

print(f"From (UTC) : {FROM_ISO}")
print(f"To   (UTC) : {TO_ISO}")
print(f"span_kind  : {SPAN_KIND}")
print(f"ml_app     : {ML_APP}")

From (UTC) : 2026-02-21T09:40:00Z
To   (UTC) : 2026-02-21T11:40:00Z
span_kind  : workflow
ml_app     : gemini-ss6_1


## 4. Query the LLMObs Export API (with pagination)

Uses `POST /api/v2/llm-obs/v1/spans/events/search` — the dedicated LLMObs endpoint.

> **Note:** Content-Type must be `application/vnd.api+json` (not `application/json`).

In [28]:
URL = f"https://api.{DD_SITE}/api/v2/llm-obs/v1/spans/events/search"

HEADERS = {
    "DD-API-KEY":          DD_API_KEY,
    "DD-APPLICATION-KEY":  DD_APP_KEY,
    "Content-Type":        "application/vnd.api+json",
    "Accept":              "application/vnd.api+json",
}

PAGE_LIMIT = 5000  # max supported by this endpoint

all_spans = []
cursor    = None
page_num  = 0

while True:
    page_num += 1
    cursor_preview = "None" if cursor is None else cursor[:24] + "..."
    print(f"  Page {page_num} (cursor={cursor_preview})...", end=" ", flush=True)

    page_block = {"limit": PAGE_LIMIT}
    if cursor:
        page_block["cursor"] = cursor

    body = {
        "data": {
            "type": "spans",
            "attributes": {
                "filter": {
                    "from":      FROM_ISO,
                    "to":        TO_ISO,
                    "span_kind": SPAN_KIND,
                    "ml_app":    ML_APP,
                },
                "page": page_block,
                "sort": "timestamp",
            },
        }
    }

    resp = requests.post(URL, headers=HEADERS, json=body, timeout=30)

    if not resp.ok:
        print(f"\n❌ HTTP {resp.status_code}: {resp.text[:300]}")
        break

    data = resp.json()
    spans_page = data.get("data") or []
    all_spans.extend(spans_page)
    print(f"got {len(spans_page)} spans  (total: {len(all_spans)})")

    # Paginate — meta.page is null when there are no more results
    meta_page  = (data.get("meta") or {}).get("page") or {}
    next_cursor = meta_page.get("after") if isinstance(meta_page, dict) else None

    if not next_cursor or len(spans_page) < PAGE_LIMIT:
        break
    cursor = next_cursor

print(f"\n✅ Total spans fetched: {len(all_spans)}")

  Page 1 (cursor=None)... got 817 spans  (total: 817)

✅ Total spans fetched: 817


## 5. Inspect a Sample Span

The LLMObs Export API returns a **clean, flat response** — `input` and `output` are direct fields, no digging through `attributes.attributes`.

In [29]:
if all_spans:
    sample = all_spans[0]
    a = sample.get("attributes", {})

    print("span_id      :", a.get("span_id"))
    print("trace_id     :", a.get("trace_id"))
    print("parent_id    :", a.get("parent_id"))
    print("name         :", a.get("name"))
    print("span_kind    :", a.get("span_kind"))
    print("ml_app       :", a.get("ml_app"))
    print("model_name   :", a.get("model_name"))
    print("model_provider:", a.get("model_provider"))
    print("status       :", a.get("status"))
    print("start_ns     :", a.get("start_ns"))
    print("duration     :", a.get("duration"), "ns")
    print("tags         :", a.get("tags"))
    print("metrics      :", a.get("metrics"))
    print("metadata     :", a.get("metadata"))
    print("\n--- input ---")
    print(json.dumps(a.get("input"), ensure_ascii=False, indent=2))
    print("\n--- output ---")
    print(json.dumps(a.get("output"), ensure_ascii=False, indent=2))
else:
    print("No spans returned. Try the debug cell below.")

span_id      : 14319689627778792714
trace_id     : 69997e7600000000c6b613368b60580e
parent_id    : undefined
name         : extract_ss61_form
span_kind    : workflow
ml_app       : gemini-ss6_1
model_name   : None
model_provider: None
status       : ok
start_ns     : 1771667062520
duration     : 22787782000 ns
tags         : ['ml_app:gemini-ss6_1', 'ddtrace.version:4.4.0', 'source:llm-obs-events-updater', 'sensitive_data_category:url', 'env:development', 'version:0.0.1', 'error:0', 'source:integration', 'sensitive_data:http_url', 'service:vote-extractor', 'language:python']
metrics      : None
metadata     : None

--- input ---
{
  "value": "{\"input_data\": {\"drive_uri\": \"[HTTP(S) URL Scanner]\", \"source_file_metadata\": {\"province_name\": \"\\u0e2b\\u0e19\\u0e2d\\u0e07\\u0e04\\u0e32\\u0e22\", \"path\": \"\\u0e1b\\u0e23\\u0e30\\u0e01\\u0e32\\u0e28\\u0e1c\\u0e25\\u0e01\\u0e32\\u0e23\\u0e19\\u0e31\\u0e1a\\u0e04\\u0e30\\u0e41\\u0e19\\u0e19 \\u0e2a\\u0e2a. (8 \\u0e01.\\u0e1e. 2569)/\

## 6. Extract Input & Output from Each Span

In [30]:
def ns_to_iso(ns) -> str | None:
    """Convert nanosecond epoch to ISO 8601 string."""
    if ns is None:
        return None
    return datetime.fromtimestamp(int(ns) / 1e9, tz=timezone.utc).isoformat()


def parse_value(val):
    """Parse a string that contains JSON (possibly with \\uXXXX Thai escapes).

    The Datadog backend serialised input/output with ensure_ascii=True, so Thai
    characters arrive as literal \\uXXXX sequences inside the string.
    json.loads() decodes those back to real Unicode codepoints.
    Fallback: return the original value if it's not a JSON string.
    """
    if val is None:
        return None
    if isinstance(val, (dict, list)):
        return val  # already parsed
    if isinstance(val, str):
        try:
            return json.loads(val)
        except (json.JSONDecodeError, ValueError):
            return val
    return val


def extract_span_record(span: dict) -> dict:
    """Extract key fields from an LLMObs Export API response item."""
    a: dict = span.get("attributes", {})

    inp = a.get("input") or {}
    out = a.get("output") or {}
    metrics: dict = a.get("metrics") or {}

    return {
        "span_id"        : a.get("span_id"),
        "trace_id"       : a.get("trace_id"),
        "parent_id"      : a.get("parent_id"),
        "name"           : a.get("name"),
        "span_kind"      : a.get("span_kind"),
        "ml_app"         : a.get("ml_app"),
        "model_name"     : a.get("model_name"),
        "model_provider" : a.get("model_provider"),
        "status"         : a.get("status"),
        "start_time"     : ns_to_iso(a.get("start_ns")),
        "duration_ms"    : round(a.get("duration", 0) / 1e6, 3) if a.get("duration") else None,
        # parse_value decodes the double-escaped \\uXXXX sequences → real Thai chars
        "input_value"    : parse_value(inp.get("value")),
        "input_messages" : inp.get("messages"),
        "output_value"   : parse_value(out.get("value")),
        "output_messages": out.get("messages"),
        "input_tokens"   : metrics.get("input_tokens"),
        "output_tokens"  : metrics.get("output_tokens"),
        "total_tokens"   : metrics.get("total_tokens"),
        "metadata"       : a.get("metadata"),
        "tags"           : a.get("tags"),
    }

In [31]:
records = [extract_span_record(s) for s in all_spans]
print(f"Extracted {len(records)} records")

Extracted 817 records


## 7. Preview as DataFrame

In [32]:
df = pd.DataFrame(records)
print(f"Shape: {df.shape}")
display_cols = [c for c in ["span_id", "trace_id", "span_kind", "name", "start_time", "duration_ms", "input_value", "output_value"] if c in df.columns]
df[display_cols].head(10)

Shape: (817, 20)


,span_id,trace_id,span_kind,name,start_time,duration_ms,input_value,output_value
0,14319689627778792714,69997e7600000000c6b613368b60580e,workflow,extract_ss61_form,1970-01-01T00:29:31.667063+00:00,22787.782,{'input_data': {'drive_uri': '[HTTP(S) URL Sca...,"{'form_info': {'form_type': 'แบ่งเขต', 'provin..."
1,2495297748776649879,69997e7600000000cb156ac38de0d75c,workflow,extract_ss61_form,1970-01-01T00:29:31.667063+00:00,26808.277,{'input_data': {'drive_uri': '[HTTP(S) URL Sca...,"{'form_info': {'form_type': 'แบ่งเขต', 'provin..."
2,3239064239954955975,69997e7600000000f6b0fccdfcb7e7f7,workflow,extract_ss61_form,1970-01-01T00:29:31.667063+00:00,26704.877,{'input_data': {'drive_uri': '[HTTP(S) URL Sca...,"{'form_info': {'form_type': 'แบ่งเขต', 'provin..."
3,12498306820193470914,69997e760000000043e970250ed33f83,workflow,extract_ss61_form,1970-01-01T00:29:31.667063+00:00,56083.239,{'input_data': {'drive_uri': '[HTTP(S) URL Sca...,"{'form_info': {'form_type': 'บัญชีรายชื่อ', 'p..."
4,14538500073484461951,69997e76000000007794bdaf22182738,workflow,extract_ss61_form,1970-01-01T00:29:31.667063+00:00,67200.796,{'input_data': {'drive_uri': '[HTTP(S) URL Sca...,"{'form_info': {'form_type': 'บัญชีรายชื่อ', 'p..."
5,859892776724979700,69997e8d00000000badfd0e75abb606e,workflow,extract_ss61_form,1970-01-01T00:29:31.667085+00:00,63543.057,{'input_data': {'drive_uri': '[HTTP(S) URL Sca...,"{'form_info': {'form_type': 'บัญชีรายชื่อ', 'p..."
6,17200132583041637687,69997e91000000008458273446b4b8c1,workflow,extract_ss61_form,1970-01-01T00:29:31.667089+00:00,25821.752,{'input_data': {'drive_uri': '[HTTP(S) URL Sca...,"{'form_info': {'form_type': 'แบ่งเขต', 'provin..."
7,2441704981399109931,69997e9100000000a7dd0e812fce156d,workflow,extract_ss61_form,1970-01-01T00:29:31.667089+00:00,32677.172,{'input_data': {'drive_uri': '[HTTP(S) URL Sca...,"{'form_info': {'form_type': 'แบ่งเขต', 'provin..."
8,15425464856325127913,69997eab000000001d9729839389006f,workflow,extract_ss61_form,1970-01-01T00:29:31.667115+00:00,29373.988,{'input_data': {'drive_uri': '[HTTP(S) URL Sca...,"{'form_info': {'form_type': 'แบ่งเขต', 'provin..."
9,6838661395927482059,69997eae00000000d2b0702cf911d1f4,workflow,extract_ss61_form,1970-01-01T00:29:31.667119+00:00,30772.205,{'input_data': {'drive_uri': '[HTTP(S) URL Sca...,"{'form_info': {'form_type': 'แบ่งเขต', 'provin..."


In [33]:
# Quick look at one record's input / output — Thai chars should now be readable
if records:
    r = records[0]

    def pretty(val):
        if val is None:
            return "null"
        if isinstance(val, (dict, list)):
            return json.dumps(val, ensure_ascii=False, indent=2)
        return str(val)

    print("=== INPUT VALUE ===")
    print(pretty(r["input_value"]))
    print("\n=== INPUT MESSAGES ===")
    print(pretty(r["input_messages"]))
    print("\n=== OUTPUT VALUE ===")
    print(pretty(r["output_value"]))
    print("\n=== OUTPUT MESSAGES ===")
    print(pretty(r["output_messages"]))

=== INPUT VALUE ===
{
  "input_data": {
    "drive_uri": "[HTTP(S) URL Scanner]",
    "source_file_metadata": {
      "province_name": "หนองคาย",
      "path": "ประกาศผลการนับคะแนน สส. (8 ก.พ. 2569)/หนองคาย/แบบแบ่งเขต/70. หนองคาย เขต 3.pdf",
      "size_mb": 0.1196,
      "file_id": "1JDOkP6nW0qNfSg27DBX_7ZzZ4epF6u32",
      "folder_id": "1EvhR0-EoN_vCkqo3R9xFsNp6ipA7YsHy",
      "form_type": "แบ่งเขต"
    }
  },
  "config": {
    "model": "gemini-2.5-pro",
    "temperature": 0.0,
    "max_tokens": 16384,
    "thinking_mode": "LOW"
  }
}

=== INPUT MESSAGES ===
null

=== OUTPUT VALUE ===
{
  "form_info": {
    "form_type": "แบ่งเขต",
    "province": "หนองคาย",
    "constituency_number": "3",
    "date": "8 กุมภาพันธ์ พ.ศ. ๒๕๖๙"
  },
  "ballot_summary": {
    "eligible_voters": {
      "arabic": 137314,
      "thai_text": "หนึ่งแสนสามหมื่นเจ็ดพันสามร้อยสิบสี่"
    },
    "present_voters": {
      "arabic": 87064,
      "thai_text": "แปดหมื่นเจ็ดพันหกสิบสี่"
    },
    "valid_ballots": {

## 8. Write to JSONL

In [34]:
OUTPUT_PATH = Path("./datasets/ss6_1_llmobs_traces.jsonl")
OUTPUT_PATH.parent.mkdir(parents=True, exist_ok=True)

with OUTPUT_PATH.open("w", encoding="utf-8") as f:
    for rec in records:
        f.write(json.dumps(rec, ensure_ascii=False, default=str) + "\n")

print(f"✅ Written {len(records)} records → {OUTPUT_PATH.resolve()}")
print(f"   File size: {OUTPUT_PATH.stat().st_size / 1024:.1f} KB")

✅ Written 817 records → /Users/nuttee.jirattivongvibul/Projects/genai-app-python/notebooks/datasets/ss6_1_llmobs_traces.jsonl
   File size: 7787.4 KB


## 9. Summary Statistics

In [35]:
print("=== Span kinds ===")
print(df["span_kind"].value_counts().to_string())

print("\n=== Span names (top 20) ===")
print(df["name"].value_counts().head(20).to_string())

print("\n=== Token usage ===")
for col in ["input_tokens", "output_tokens", "total_tokens"]:
    if col in df.columns:
        s = pd.to_numeric(df[col], errors="coerce")
        print(f"  {col}: sum={s.sum():.0f}  mean={s.mean():.1f}  max={s.max():.0f}")

print("\n=== Records with input & output ===")
has_in  = df["input_value"].notna() | df["input_messages"].notna()
has_out = df["output_value"].notna() | df["output_messages"].notna()
print(f"  input  : {has_in.sum()} / {len(df)}")
print(f"  output : {has_out.sum()} / {len(df)}")
print(f"  both   : {(has_in & has_out).sum()} / {len(df)}")

=== Span kinds ===
span_kind
workflow    817

=== Span names (top 20) ===
name
extract_ss61_form    817

=== Token usage ===
  input_tokens: sum=0  mean=nan  max=nan
  output_tokens: sum=0  mean=nan  max=nan
  total_tokens: sum=0  mean=nan  max=nan

=== Records with input & output ===
  input  : 817 / 817
  output : 816 / 817
  both   : 816 / 817


---

## 10. Party Number Accuracy Analysis

The official party numbers for SS6-1 (2026 election) are **fixed nationally** — every constituency's party-list ballot has the same numbers 1–52.

This section checks whether the LLM extracted the **correct number** for each party, and categorises the error pattern (exact, off-by-1, off-by-2, larger shift).


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from pathlib import Path

matplotlib.rcParams['font.family'] = ['Tahoma', 'DejaVu Sans']

# ── Official party number → canonical name ──────────────────────────────────
OFFICIAL = {
     1: 'ไทยทรัพย์ทวี',          2: 'เพื่อชาติไทย',
     3: 'ใหม่',                    4: 'มิติใหม่',
     5: 'รวมใจไทย',               6: 'รวมไทยสร้างชาติ',
     7: 'พลวัต',                   8: 'ประชาธิปไตยใหม่',
     9: 'เพื่อไทย',               10: 'ทางเลือกใหม่',
    11: 'เศรษฐกิจ',               12: 'เสรีรวมไทย',
    13: 'รวมพลังประชาชน',         14: 'ท้องที่ไทย',
    15: 'อนาคตไทย',               16: 'พลังเพื่อไทย',
    17: 'ไทยชนะ',                 18: 'พลังสังคมใหม่',
    19: 'สังคมประชาธิปไตยไทย',   20: 'ฟิวชัน',
    21: 'ไทยรวมพลัง',             22: 'ก้าวอิสระ',
    23: 'ปวงชนไทย',               24: 'วินัยไทย',
    25: 'เพื่อชีวิตใหม่',        26: 'คลองไทย',
    27: 'ประชาธิปัตย์',           28: 'ไทยก้าวหน้า',
    29: 'ไทยภักดี',               30: 'แรงงานสร้างชาติ',
    31: 'ประชากรไทย',             32: 'ครูไทยเพื่อประชาชน',
    33: 'ประชาชาติ',               34: 'สร้างอนาคตไทย',
    35: 'รักชาติ',                 36: 'ไทยพร้อม',
    37: 'ภูมิใจไทย',              38: 'พลังธรรมใหม่',
    39: 'กรีน',                    40: 'ไทยธรรม',
    41: 'แผ่นดินธรรม',            42: 'กล้าธรรม',
    43: 'พลังประชารัฐ',           44: 'โอกาสไทย',
    45: 'เป็นธรรม',               46: 'ประชาชน',
    47: 'ประชาไทย',               48: 'ไทยสร้างไทย',
    49: 'ไทยก้าวใหม่',            50: 'ประชาอาสาชาติ',
    51: 'พลัง',                    52: 'เครือข่ายชาวนาแห่งประเทศไทย',
}

# Reverse map: canonical name → official number
OFFICIAL_BY_NAME = {v: k for k, v in OFFICIAL.items()}

# Aliases: LLM party name variants → canonical name
# (handles 'พรรค' prefix, typos, OCR variants)
ALIASES = {
    # with พรรค prefix
    **{f'พรรค{v}': v for v in OFFICIAL.values()},
    # common OCR / LLM misspellings
    'วิชชั่นใหม่':          'วินัยไทย',
    'วิชชันใหม่':           'วินัยไทย',
    'วิชชั่นไทย':           'วินัยไทย',
    'วิชั่นใหม่':           'วินัยไทย',
    'รักษ์ถิ่นใหม่':        'วินัยไทย',
    'พรรควิชชั่นใหม่':      'วินัยไทย',
    'พรรควิชชันใหม่':       'วินัยไทย',
    'พรรควิชชั่นไทย':       'วินัยไทย',
    'พรรควินัยไทย':         'วินัยไทย',
    'โอกาสใหม่':            'โอกาสไทย',
    'พรรคโอกาสใหม่':        'โอกาสไทย',
    'โอกาสไทย':             'โอกาสไทย',
    'พร้อม':                'พลัง',
    'พรรคพร้อม':            'พลัง',
    'พรรคพลัง':             'พลัง',
    'ท้องทีไทย':            'ท้องที่ไทย',
    'ท้องถิ่นไทย':          'ท้องที่ไทย',
    'พรรคท้องถิ่นไทย':      'ท้องที่ไทย',
    'นิติใหม่':             'มิติใหม่',
    'พลวัด':               'พลวัต',
    'พลวัติ':               'พลวัต',
    'กลาธรรม':              'กล้าธรรม',
    'ประชาติปไตยใหม่':      'ประชาธิปไตยใหม่',
    'ประชาธิปปัตย์':        'ประชาธิปัตย์',
    'ประชาธิปไตยไทยใหม่':   'ประชาธิปไตยใหม่',
    'ฟิวชั่น':              'ฟิวชัน',
    'พรรคฟิวชั่น':          'ฟิวชัน',
    'ไทรวมพลัง':            'ไทยรวมพลัง',
    'เก้าอิสระ':            'ก้าวอิสระ',
    'สังคมประชาธิปไตย':     'สังคมประชาธิปไตยไทย',
    'สังคมประชาธิปไตยฯ':    'สังคมประชาธิปไตยไทย',
    'พรรคสังคมประชาธิปไตย': 'สังคมประชาธิปไตยไทย',
    'พรรคสังคมใหม่':        'พลังสังคมใหม่',
    'พลังสังคมไทย':         'พลังสังคมใหม่',
    'พรรคพลังสังคมไทย':     'พลังสังคมใหม่',
    'พลังสังคมใหม่ /':      'พลังสังคมใหม่',
    'ไทยพิทักษ์ธรรม':       'เครือข่ายชาวนาแห่งประเทศไทย',  # mapped by position? skip
    'เศรฐกิจ':              'เศรษฐกิจ',
    'เศรษฐกิจไทย':          'เศรษฐกิจ',
    'เศรษฐกิจ/':            'เศรษฐกิจ',
    'รวมคะแนนทั้งสิ้น':     None,  # artefact row
    'ไทยสร้างชาติ':         'ไทยสร้างไทย',  # note: ไทยสร้างไทย vs ไทยสร้างชาติ differ
    'สรางอนาคตไทย':         'สร้างอนาคตไทย',
    'แรงงานสร้างไทย':       'แรงงานสร้างชาติ',
    'พลังธรรมไทย':          'พลังธรรมใหม่',
    'ประชาธิปัตย์':         'ประชาธิปัตย์',
    'บ่วงชนไทย':            'ปวงชนไทย',
    'ลองไทย':               'คลองไทย',
    'พรรคลองไทย':           'คลองไทย',
    'อนาคตใหม่':            'อนาคตไทย',
    'รักษ์ชาติ':            'รักชาติ',
    'ไทยขนะ':               'ไทยชนะ',
    'ไทยกาวใหม่':           'ไทยก้าวใหม่',
    'เพื่อบ้านใหม่':        'เพื่อบ้านเมือง',  # beyond official 52 — keep name
    'ไทยพิทักษ์ธรรม':       None,  # บอร์ 53-54, beyond official 52
    'พรรคเครือข่ายชาวนา':   'เครือข่ายชาวนาแห่งประเทศไทย',
    'เครือข่ายชาวนาฯ':      'เครือข่ายชาวนาแห่งประเทศไทย',
    'เครือขายชาวนาแหงประเทศไทย': 'เครือข่ายชาวนาแห่งประเทศไทย',
    'เครือขายชาวนาแห่งประเทศไทย': 'เครือข่ายชาวนาแห่งประเทศไทย',
    'เครือข่าวชาวนาแห่งประเทศไทย': 'เครือข่ายชาวนาแห่งประเทศไทย',
    'พรรคเครือข่ายชาวนาแห่งประเทศไทย': 'เครือข่ายชาวนาแห่งประเทศไทย',
    'ไทยสร้างไทย-':         'ไทยสร้างไทย',
    'เพื่อบ้านเมือง -':     'เพื่อบ้านเมือง',  # beyond 52
    'พลังไทยรักชาติ -':     'พลังไทยรักชาติ',  # beyond 52
    'ไทยสร้างชาติ':         'รวมไทยสร้างชาติ',
}


def resolve_party(name):
    """Resolve an LLM party name to its canonical name (or None if unknown)."""
    if name in OFFICIAL_BY_NAME:
        return name
    return ALIASES.get(name)


# ── Load LLM party-list CSV ──────────────────────────────────────────────────
ASSETS = Path('assets/killernay')
llm_pl = pd.read_csv(ASSETS / 'llm_party_list.csv')
llm_pl.columns = ['province_code', 'province', 'constituency', 'number_llm', 'party_raw', 'votes']
llm_pl['number_llm'] = pd.to_numeric(llm_pl['number_llm'], errors='coerce').astype('Int64')

# Resolve each party name → canonical name → official number
llm_pl['party_canonical']   = llm_pl['party_raw'].map(resolve_party)
llm_pl['number_official']   = llm_pl['party_canonical'].map(OFFICIAL_BY_NAME).astype('Int64')
llm_pl['number_error']      = (llm_pl['number_llm'] - llm_pl['number_official']).astype('Int64')
llm_pl['number_error_abs']  = llm_pl['number_error'].abs()

resolved   = llm_pl['party_canonical'].notna()
in_official = llm_pl['number_official'].notna()

print(f'Total LLM party-list rows : {len(llm_pl):,}')
print(f'Resolved to canonical name : {resolved.sum():,}  ({resolved.mean()*100:.1f}%)')
print(f'Mapped to official number  : {in_official.sum():,}  ({in_official.mean()*100:.1f}%)')
print(f'Unresolved party names     : {(~resolved).sum():,}')


In [ ]:
# ── Error distribution ───────────────────────────────────────────────────────
matched = llm_pl[in_official].copy()

def classify_error(e):
    if pd.isna(e):   return 'no official'
    e = int(e)
    if e == 0:       return '✅ correct'
    if e == +1:      return '+1 (LLM high)'
    if e == -1:      return '-1 (LLM low)'
    if e == +2:      return '+2'
    if e == -2:      return '-2'
    if abs(e) <= 5:  return f'{e:+d} (small shift)'
    return           f'large shift ({e:+d})'

matched['error_class'] = matched['number_error'].map(classify_error)

error_dist = (
    matched['error_class'].value_counts()
    .rename_axis('error_class')
    .reset_index(name='count')
)
error_dist['pct'] = (error_dist['count'] / len(matched) * 100).round(1)

print('=== Party Number Error Distribution ===')
print(error_dist.to_string(index=False))
print(f'\nBase: {len(matched):,} rows with a resolvable official number')


In [ ]:
# ── Per-party number accuracy ────────────────────────────────────────────────
party_accuracy = (
    matched.groupby('party_canonical')
    .agg(
        official_number = ('number_official', 'first'),
        total_rows      = ('number_llm', 'count'),
        correct         = ('number_error', lambda x: (x == 0).sum()),
        most_common_llm_num = ('number_llm', lambda x: x.mode().iloc[0] if len(x) else None),
        error_values    = ('number_error', lambda x: sorted(x.dropna().astype(int).unique().tolist())),
    )
    .assign(
        accuracy_pct = lambda d: (d['correct'] / d['total_rows'] * 100).round(1),
        wrong        = lambda d: d['total_rows'] - d['correct'],
    )
    .sort_values(['accuracy_pct', 'official_number'])
)
party_accuracy.index.name = 'party'

print('=== Per-Party Number Accuracy (worst first) ===')
from IPython.display import display
display(party_accuracy[[
    'official_number', 'total_rows', 'correct', 'wrong',
    'accuracy_pct', 'most_common_llm_num', 'error_values'
]])


In [ ]:
# ── Visualise: accuracy heatmap by official number ───────────────────────────
fig, axes = plt.subplots(1, 2, figsize=(16, 7))
fig.suptitle('Party Number Accuracy — LLM vs Official', fontsize=13)

# Left: accuracy % per party (sorted by official number)
ax = axes[0]
sorted_by_num = party_accuracy.sort_values('official_number')
colors = ['#2ecc71' if v >= 95 else '#e67e22' if v >= 75 else '#e74c3c'
          for v in sorted_by_num['accuracy_pct']]
bars = ax.barh(
    range(len(sorted_by_num)),
    sorted_by_num['accuracy_pct'],
    color=colors, edgecolor='white'
)
ax.set_yticks(range(len(sorted_by_num)))
ax.set_yticklabels(
    [f'{int(row.official_number):02d}. {idx}' for idx, row in sorted_by_num.iterrows()],
    fontsize=7
)
ax.set_xlabel('Correct number rate (%)')
ax.set_title('Correct party number rate\n(green ≥95%, orange ≥75%, red <75%)')
ax.set_xlim(0, 110)
for i, v in enumerate(sorted_by_num['accuracy_pct']):
    ax.text(v + 1, i, f'{v:.0f}%', va='center', fontsize=6)

# Right: number error distribution bar chart
ax = axes[1]
err_counts = matched['number_error'].dropna().astype(int).value_counts().sort_index()
ax.bar(err_counts.index.astype(str), err_counts.values,
       color=['#2ecc71' if e == 0 else '#e74c3c' for e in err_counts.index])
ax.set_xlabel('LLM number − Official number')
ax.set_ylabel('Row count')
ax.set_title('Distribution of party number errors\n(0 = correct)')
ax.tick_params(axis='x', rotation=45, labelsize=7)

plt.tight_layout()
plt.savefig(ASSETS / 'party_number_accuracy.png', dpi=120, bbox_inches='tight')
plt.show()


In [ ]:
# ── Unresolved party names (not in official list or alias map) ───────────────
unresolved = (
    llm_pl[~resolved]
    .groupby('party_raw')
    .agg(rows=('number_llm', 'count'),
         numbers_used=('number_llm', lambda x: sorted(x.dropna().astype(int).unique().tolist())))
    .sort_values('rows', ascending=False)
)
print(f'Unresolved party names ({len(unresolved)} unique):')
display(unresolved)

# ── Rows where LLM extracted number > 52 (beyond official range) ─────────────
beyond = llm_pl[llm_pl['number_llm'] > 52]
print(f'\nRows with party number > 52 (beyond official range): {len(beyond):,}')
display(
    beyond.groupby(['party_raw', 'number_llm'])
    .size().reset_index(name='count')
    .sort_values(['number_llm', 'count'], ascending=[True, False])
    .reset_index(drop=True)
)


---

## 11. Hot-Fix JSONL

Applies two corrections directly to `ss6_1_llmobs_traces.jsonl`:

1. **Constituency number** — extracted from the file path (`เขต N`) which is always correct; overwrites any wrong value in `form_info.constituency_number`
2. **Party numbers** (บัญชีรายชื่อ only) — each party name is resolved to the official number (1–52) via the alias map; corrects the +1 shift and OCR errors

The original file is backed up as `ss6_1_llmobs_traces.jsonl.bak` before writing.


In [ ]:
import json, re, shutil
from pathlib import Path

DATASETS   = Path('datasets')
JSONL_PATH = DATASETS / 'ss6_1_llmobs_traces.jsonl'
BAK_PATH   = DATASETS / 'ss6_1_llmobs_traces.jsonl.bak'

THAI = str.maketrans('๐๑๒๓๔๕๖๗๘๙', '0123456789')

OFFICIAL = {
     1:'ไทยทรัพย์ทวี',    2:'เพื่อชาติไทย',     3:'ใหม่',
     4:'มิติใหม่',         5:'รวมใจไทย',          6:'รวมไทยสร้างชาติ',
     7:'พลวัต',            8:'ประชาธิปไตยใหม่',   9:'เพื่อไทย',
    10:'ทางเลือกใหม่',    11:'เศรษฐกิจ',         12:'เสรีรวมไทย',
    13:'รวมพลังประชาชน',  14:'ท้องที่ไทย',       15:'อนาคตไทย',
    16:'พลังเพื่อไทย',    17:'ไทยชนะ',           18:'พลังสังคมใหม่',
    19:'สังคมประชาธิปไตยไทย',20:'ฟิวชัน',        21:'ไทยรวมพลัง',
    22:'ก้าวอิสระ',        23:'ปวงชนไทย',         24:'วินัยไทย',
    25:'เพื่อชีวิตใหม่',  26:'คลองไทย',          27:'ประชาธิปัตย์',
    28:'ไทยก้าวหน้า',      29:'ไทยภักดี',         30:'แรงงานสร้างชาติ',
    31:'ประชากรไทย',       32:'ครูไทยเพื่อประชาชน',33:'ประชาชาติ',
    34:'สร้างอนาคตไทย',   35:'รักชาติ',           36:'ไทยพร้อม',
    37:'ภูมิใจไทย',        38:'พลังธรรมใหม่',      39:'กรีน',
    40:'ไทยธรรม',          41:'แผ่นดินธรรม',       42:'กล้าธรรม',
    43:'พลังประชารัฐ',     44:'โอกาสไทย',          45:'เป็นธรรม',
    46:'ประชาชน',           47:'ประชาไทย',           48:'ไทยสร้างไทย',
    49:'ไทยก้าวใหม่',      50:'ประชาอาสาชาติ',      51:'พลัง',
    52:'เครือข่ายชาวนาแห่งประเทศไทย',
}
OFFICIAL_BY_NAME = {v: k for k, v in OFFICIAL.items()}

ALIASES = {
    **{f'พรรค{v}': v for v in OFFICIAL.values()},
    'วิชชั่นใหม่':'วินัยไทย',   'วิชชันใหม่':'วินัยไทย',   'วิชชั่นไทย':'วินัยไทย',
    'วิชั่นใหม่':'วินัยไทย',   'รักษ์ถิ่นใหม่':'วินัยไทย',
    'พรรควิชชันใหม่':'วินัยไทย', 'พรรควิชชั่นใหม่':'วินัยไทย',
    'โอกาสใหม่':'โอกาสไทย', 'โอกาสใหม่/':'โอกาสไทย', 'พรรคโอกาสใหม่':'โอกาสไทย',
    'พร้อม':'พลัง',           'พรรคพร้อม':'พลัง',
    'พลวัด':'พลวัต',     'พลวัติ':'พลวัต',  'พรรคพลวัติ':'พลวัต',
    'นิติใหม่':'มิติใหม่', 'กลาธรรม':'กล้าธรรม',
    'ท้องทีไทย':'ท้องที่ไทย', 'ท้องถิ่นไทย':'ท้องที่ไทย',
    'พรรคท้องถิ่นไทย':'ท้องที่ไทย',
    'ประชาติปไตยใหม่':'ประชาธิปไตยใหม่',
    'ประชาธิปปัตย์':'ประชาธิปัตย์', 'ประชาธิปไตยไทยใหม่':'ประชาธิปไตยใหม่',
    'ฟิวชั่น':'ฟิวชัน', 'พรรคฟิวชั่น':'ฟิวชัน',
    'ไทรวมพลัง':'ไทยรวมพลัง', 'เก้าอิสระ':'ก้าวอิสระ',
    'พรรคไทรวมพลัง':'ไทยรวมพลัง',
    'สังคมประชาธิปไตย':'สังคมประชาธิปไตยไทย',
    'สังคมประชาธิปไตยฯ':'สังคมประชาธิปไตยไทย',
    'พรรคสังคมประชาธิปไตย':'สังคมประชาธิปไตยไทย',
    'พรรคสังคมใหม่':'พลังสังคมใหม่', 'พลังสังคมไทย':'พลังสังคมใหม่',
    'พรรคพลังสังคมไทย':'พลังสังคมใหม่', 'พลังสังคมใหม่ /':'พลังสังคมใหม่',
    'เศรฐกิจ':'เศรษฐกิจ', 'เศรษฐกิจไทย':'เศรษฐกิจ', 'เศรษฐกิจ/':'เศรษฐกิจ',
    'สรางอนาคตไทย':'สร้างอนาคตไทย', 'สรางอนาคตไทย':'สร้างอนาคตไทย',
    'แรงงานสร้างไทย':'แรงงานสร้างชาติ',
    'พลังธรรมไทย':'พลังธรรมใหม่', 'บ่วงชนไทย':'ปวงชนไทย',
    'ลองไทย':'คลองไทย', 'พรรคลองไทย':'คลองไทย',
    'อนาคตใหม่':'อนาคตไทย', 'รักษ์ชาติ':'รักชาติ',
    'ไทยขนะ':'ไทยชนะ', 'ไทยกาวใหม่':'ไทยก้าวใหม่',
    'เครือข่ายชาวนาฯ':'เครือข่ายชาวนาแห่งประเทศไทย',
    'เครือขายชาวนาแหงประเทศไทย':'เครือข่ายชาวนาแห่งประเทศไทย',
    'เครือขายชาวนาแห่งประเทศไทย':'เครือข่ายชาวนาแห่งประเทศไทย',
    'เครือข่าวชาวนาแห่งประเทศไทย':'เครือข่ายชาวนาแห่งประเทศไทย',
    'พรรคเครือข่ายชาวนา':'เครือข่ายชาวนาแห่งประเทศไทย',
    'ไทยสร้างชาติ':'รวมไทยสร้างชาติ', 'พรรคไทยสร้างชาติ':'รวมไทยสร้างชาติ',
    'ไทยสร้างไทย-':'ไทยสร้างไทย',
    'รวมพลังประชาชน/':'รวมพลังประชาชน', 'รวมพลังประชาชาติ':'รวมพลังประชาชน',
    'ครูไทยเพื่อปวงชน':'ครูไทยเพื่อประชาชน',
}


def resolve_official_number(party_name):
    if party_name in OFFICIAL_BY_NAME:
        return OFFICIAL_BY_NAME[party_name]
    canonical = ALIASES.get(party_name)
    return OFFICIAL_BY_NAME.get(canonical) if canonical else None


shutil.copy2(JSONL_PATH, BAK_PATH)
print(f'Backed up → {BAK_PATH}')

con_fixes, party_fixes = 0, 0
unresolved = {}
fixed_lines = []

with open(JSONL_PATH) as f:
    for line in f:
        t = json.loads(line)
        iv, ov = t.get('input_value', {}), t.get('output_value', {})
        if not ov:
            fixed_lines.append(json.dumps(t, ensure_ascii=False))
            continue

        meta      = iv.get('input_data', {}).get('source_file_metadata', {})
        path      = meta.get('path', '')
        form_type = meta.get('form_type', '')

        # Fix 1: constituency number from path
        m = re.search(r'เขต\s*(\d+)', path)
        if m:
            correct_con = int(m.group(1))
            raw = ov.get('form_info', {}).get('constituency_number', '')
            try:
                llm_con = int(str(raw).translate(THAI))
            except Exception:
                llm_con = None
            if llm_con != correct_con:
                ov['form_info']['constituency_number'] = str(correct_con)
                con_fixes += 1

        # Fix 2: party numbers via official map
        if form_type == 'บัญชีรายชื่อ':
            for result in ov.get('results', []):
                official_no = resolve_official_number(result.get('party_name', ''))
                if official_no is not None and result.get('number') != official_no:
                    result['number'] = official_no
                    party_fixes += 1
                elif official_no is None:
                    n = result.get('party_name', '')
                    unresolved[n] = unresolved.get(n, 0) + 1

        t['output_value'] = ov
        fixed_lines.append(json.dumps(t, ensure_ascii=False))

with open(JSONL_PATH, 'w') as f:
    f.write('\n'.join(fixed_lines) + '\n')

print(f'\n✅ Hot-fix complete')
print(f'   Constituency number fixes : {con_fixes}')
print(f'   Party number fixes        : {party_fixes}')
print(f'   Still unresolved          : {len(unresolved)} unique names, {sum(unresolved.values())} total rows')
if unresolved:
    print('\nTop unresolved (likely non-party PDF rows):')
    for k, v in sorted(unresolved.items(), key=lambda x: -x[1])[:10]:
        print(f'  {v:4d}  {repr(k)}')
print(f'\nOutput : {JSONL_PATH}')
print(f'Backup : {BAK_PATH}')


---

## 12. Export Fixed JSONL → killernay CSV Format

Converts `ss6_1_llmobs_traces.jsonl` (after all hot-fixes) into the same column layout as the killernay ground-truth CSVs so they can be compared directly in `ss6-1-killernay-comparison.ipynb`.

| Output file | Columns |
|---|---|
| `assets/killernay/llm_constituency.csv` | รหัสจังหวัด, จังหวัด, เขต, หมายเลข, ชื่อผู้สมัคร, พรรค, คะแนน |
| `assets/killernay/llm_party_list.csv` | รหัสจังหวัด, จังหวัด, เขต, หมายเลข, พรรค, คะแนน |


In [38]:
import csv, json
from pathlib import Path

ASSETS     = Path('assets/killernay')
DATASETS   = Path('datasets')
JSONL_PATH = DATASETS / 'ss6_1_llmobs_traces.jsonl'

# ── Build province → code map from killernay ground truth ────────────────────
PROVINCE_CODE: dict[str, str] = {}
for csv_file in [ASSETS / 'constituency.csv', ASSETS / 'party_list.csv']:
    with open(csv_file, encoding='utf-8-sig') as f:
        for row in csv.DictReader(f):
            PROVINCE_CODE[row['จังหวัด']] = row['รหัสจังหวัด']

print(f'Province code map: {len(PROVINCE_CODE)} provinces loaded')

# ── Parse JSONL → two row collections ──────────────────────────────────
# Party-list uses a dict keyed by (province, constituency, number) so that:
#   - phantom rows (number > 52) are dropped
#   - duplicate rows (same constituency processed twice, or LLM double-read)
#     are collapsed by keeping the row with the highest vote count
con_rows: list[dict] = []
pl_best:  dict       = {}   # (province, constituency, number) → row

pl_phantom = pl_dedup = skipped = 0

with open(JSONL_PATH) as f:
    for line in f:
        t = json.loads(line)
        iv  = t.get('input_value', {})
        ov  = t.get('output_value', {})
        if not ov:
            skipped += 1
            continue

        meta      = iv.get('input_data', {}).get('source_file_metadata', {})
        form_type = meta.get('form_type', '')
        fi        = ov.get('form_info', {})

        province = fi.get('province', meta.get('province_name', ''))
        try:
            constituency = int(fi.get('constituency_number', 0))
        except (ValueError, TypeError):
            constituency = 0
        province_code = PROVINCE_CODE.get(province, '')

        for result in ov.get('results', []):
            number     = result.get('number')
            vote_count = result.get('vote_count', {})
            votes      = vote_count.get('arabic', 0) if isinstance(vote_count, dict) else 0

            if form_type == 'แบ่งเขต':
                con_rows.append({
                    'รหัสจังหวัด':  province_code,
                    'จังหวัด':      province,
                    'เขต':          constituency,
                    'หมายเลข':      number,
                    'ชื่อผู้สมัคร': result.get('candidate_name', ''),
                    'พรรค':         result.get('party_name', ''),
                    'คะแนน':        votes,
                })

            elif form_type == 'บัญชีรายชื่อ':
                # Drop phantom rows (number > 52 are non-party PDF artefacts)
                if not isinstance(number, int) or number < 1 or number > 52:
                    pl_phantom += 1
                    continue

                key = (province, constituency, number)
                row = {
                    'รหัสจังหวัด': province_code,
                    'จังหวัด':    province,
                    'เขต':        constituency,
                    'หมายเลข':    number,
                    'พรรค':       result.get('party_name', ''),
                    'คะแนน':      votes,
                }
                if key in pl_best:
                    pl_dedup += 1
                    if votes > pl_best[key]['คะแนน']:   # keep higher vote count
                        pl_best[key] = row
                else:
                    pl_best[key] = row

# Sort party-list rows for a consistent output order
pl_rows = sorted(pl_best.values(), key=lambda r: (r['จังหวัด'], r['เขต'], r['หมายเลข']))

print(f'Constituency rows    : {len(con_rows):,}')
print(f'Party-list rows      : {len(pl_rows):,}')
print(f'Phantom dropped      : {pl_phantom:,}  (number > 52)')
print(f'Duplicates collapsed : {pl_dedup:,}  (kept max votes)')
print(f'Skipped (no output)  : {skipped}')

# ── Write CSVs ──────────────────────────────────────────────────────
CON_COLS = ['รหัสจังหวัด', 'จังหวัด', 'เขต', 'หมายเลข', 'ชื่อผู้สมัคร', 'พรรค', 'คะแนน']
PL_COLS  = ['รหัสจังหวัด', 'จังหวัด', 'เขต', 'หมายเลข', 'พรรค', 'คะแนน']

con_out = ASSETS / 'llm_constituency.csv'
pl_out  = ASSETS / 'llm_party_list.csv'

with open(con_out, 'w', newline='', encoding='utf-8-sig') as f:
    w = csv.DictWriter(f, fieldnames=CON_COLS)
    w.writeheader()
    w.writerows(con_rows)

with open(pl_out, 'w', newline='', encoding='utf-8-sig') as f:
    w = csv.DictWriter(f, fieldnames=PL_COLS)
    w.writeheader()
    w.writerows(pl_rows)

print(f'\n✅ Written:')
print(f'   {con_out}  ({len(con_rows):,} rows)')
print(f'   {pl_out}  ({len(pl_rows):,} rows)')


Province code map: 79 provinces loaded
Constituency rows : 3,368
Party-list rows   : 21,723
Skipped (no output): 1

✅ Written:
   assets/killernay/llm_constituency.csv  (3,368 rows)
   assets/killernay/llm_party_list.csv  (21,723 rows)


---

## 13. Data Quality Issues Found & Manual Fixes Applied

After extracting all 775 traces from Datadog LLMObs into `ss6_1_llmobs_traces.jsonl`, a review of the `output_value` fields revealed **four systematic issues**. Each was fixed by direct JSONL patch (see Section 11 for the hot-fix cell).

---

### Issue 1 — Thai Numerals Not Converted to Arabic

**Affected fields:** `form_info.constituency_number`, `form_info.date`, `officials[].position`  
**Root cause:** The schema defined `constituency_number` and `date` as `STRING` with no format constraint, so the LLM returned whatever was literally on the PDF — sometimes Thai digits.

| Field | Example (raw) | Fixed |
|---|---|---|
| `constituency_number` | `"๑๖"` | `"16"` |
| `date` | `"๘ เดือน กุมภาพันธ์ พ.ศ. ๒๕๖๙"` | `"8 เดือน กุมภาพันธ์ พ.ศ. 2569"` |
| `officials[].position` | `"...เขตเลือกตั้งที่ ๓ จังหวัด..."` | `"...เขตเลือกตั้งที่ 3 จังหวัด..."` |

**Fix applied:** Recursive `str.translate(THAI→Arabic)` across all string values in `output_value`. **540 traces** updated, **0 remaining** with Thai digits.

---

### Issue 2 — Wrong Constituency Number (LLM Hallucination)

**Affected field:** `form_info.constituency_number`  
**Root cause:** The LLM sometimes reads the wrong constituency number from the PDF header (e.g., reads เขต 4 when the document is actually เขต 5). The file **path** in `source_file_metadata` always contains the correct number.

| Province | Path says | LLM extracted | Error |
|---|---|---|---|
| สุรินทร์ | เขต 5 | 4 | off by 1 |
| สมุทรปราการ | เขต 8 | 4 | large error |
| สมุทรปราการ | เขต 6 | 5 | off by 1 |

**Fix applied:** Regex `เขต\s*(\d+)` on path → overwrite `constituency_number`. **19 traces** corrected.

---

### Issue 3 — Party-List Results Contain `candidate_name` (Schema Leak)

**Affected field:** `results[].candidate_name` in บัญชีรายชื่อ traces  
**Root cause:** The schema and prompt defined `candidate_name` as optional for both form types. For party-list forms, this field has no meaning — but the LLM populated it with the **party name** (copying it into the wrong field).

```json
// Before fix (บัญชีรายชื่อ, number=46)
{"number": 46, "candidate_name": "ประชาชาติ", "party_name": "สร้างอนาคตไทย", "vote_count": {...}}

// After fix
{"number": 46, "party_name": "ประชาชน", "vote_count": {...}}
```

**Fix applied:** `candidate_name` removed from all บัญชีรายชื่อ result rows. **382 traces / 18,789 rows** updated.

---

### Issue 4 — Wrong `party_name` in Party-List Results

**Affected field:** `results[].party_name` in บัญชีรายชื่อ traces  
**Root cause:** Party names on the PDF are often printed in small Thai text and the LLM misreads them — especially parties with similar names. However, the `number` column (party slot number 1–52) is reliably read because it is a printed numeral.

**Analysis (Section 10)** found that the dominant error was a **+1 shift**: the LLM assigns the number printed next to one party to the *next* party's name. For example, ประชาธิปัตย์ (official #27) was given number 28.

**Fix applied:** Trust `number` (1–52); look up the official party name from the national party-number map and overwrite `party_name`. Rows with number > 52 (phantom PDF rows) were left as-is. **382 traces / 18,789 rows** updated.

---

### Fix Timeline Summary

| Step | Fix | Traces affected |
|---|---|---|
| 1 | Thai → Arabic numeral conversion (all `output_value` strings) | 540 |
| 2 | Constituency number from file path | 19 |
| 3 | Remove `candidate_name` from บัญชีรายชื่อ results | 382 |
| 4 | Correct `party_name` from official number map | 382 |


---

## 14. LLM Prompt & Schema Improvement Suggestions

Based on the four data quality issues above, the following changes to the prompt and Gemini JSON schema in `ss6-1-extraction-experiments.ipynb` would prevent or reduce each class of error in future runs.

---

### Fix 1 — Force Arabic Numerals in the Prompt

Add an explicit rule under **EXTRACTION INSTRUCTIONS** so the LLM always converts Thai digits before writing to any field:

```diff
 EXTRACTION INSTRUCTIONS:
+
+ NUMERAL RULE: Always output numbers as Arabic digits (0-9), never Thai digits (๐-๙).
+   Convert any Thai digit you read from the document before writing it to the JSON.
```

Alternatively, change `constituency_number` schema type from `STRING` → `INTEGER` and `date` to a fixed format like `"8 February 2569"` — both prevent Thai numeral output.

---

### Fix 2 — Use Path Metadata to Ground Constituency Number

The constituency number is already available in `source_file_metadata.path` (e.g., `สมุทรปราการ เขต 8 (บช).pdf`). Inject it into the prompt as a hint:

```diff
 1. HEADER (form_info):
    - constituency_number: the เขตเลือกตั้งที่ number as a string
+     IMPORTANT: The file is named "{province} เขต {N}" — use N as the constituency number.
+     If the document header shows a different number, trust the file name.
```

In code, inject `N` into the prompt string at call time from `source_file_metadata.path`.

---

### Fix 3 — Suppress `candidate_name` for Party-List Forms

Send a **form-type-aware prompt** rather than a single generic prompt. For บัญชีรายชื่อ, either:

**Option A — Separate schema per form type (recommended):**
```python
# For บัญชีรายชื่อ, remove candidate_name from results items entirely
PL_RESULTS_ITEM = {
    "type": "OBJECT",
    "required": ["number", "party_name", "vote_count"],
    "properties": {
        "number":     {"type": "INTEGER"},
        "party_name": {"type": "STRING"},
        "vote_count": _num_text_pair,
        # candidate_name intentionally omitted
    },
}
```

**Option B — Explicit null instruction in prompt:**
```diff
 3. RESULTS TABLE:
    - candidate_name: full name — แบ่งเขต ONLY (leave null for บัญชีรายชื่อ)
+   For บัญชีรายชื่อ: DO NOT include candidate_name in any result row. Omit the field entirely.
```

---

### Fix 4 — Ground Party Name to Official Number in Schema

Since `number` (1–52) is reliably read and maps 1-to-1 to an official party name, the cleanest fix is to **remove `party_name` from the schema for บัญชีรายชื่อ** and derive it deterministically from `number` post-extraction:

```python
OFFICIAL_PARTY = {
    1: 'ไทยทรัพย์ทวี', 2: 'เพื่อชาติไทย', 3: 'ใหม่', # ... up to 52
}

for result in output['results']:
    result['party_name'] = OFFICIAL_PARTY.get(result['number'], result.get('party_name'))
```

If you still want the LLM to extract `party_name` as a cross-check, add an `enum` constraint to the schema:

```python
"party_name": {
    "type": "STRING",
    "enum": list(OFFICIAL_PARTY.values()),  # 52 official names
    "description": "Must exactly match one of the 52 official SS6-1 party names",
}
```

---

### Fix 5 — Add `constituency_number` Schema Type as INTEGER

Currently `constituency_number` is `STRING`. Changing to `INTEGER` forces Arabic output and prevents Thai numeral strings:

```diff
 "constituency_number": {
-    "type": "STRING",
+    "type": "INTEGER",
     "description": "Constituency zone number (เขตเลือกตั้งที่)",
 },
```

---

### Summary Table

| Issue | Recommended Fix | Impact |
|---|---|---|
| Thai digits in output | Add NUMERAL RULE to prompt + change `constituency_number` to `INTEGER` | Eliminates post-processing step |
| Wrong constituency number | Inject path-derived N into prompt as grounding hint | Eliminates 19+ errors/run |
| `candidate_name` in party-list | Use form-type-specific schema (omit field) | Eliminates schema leak |
| Wrong `party_name` | Remove from schema; derive from `number` via `OFFICIAL_PARTY` map | 100% accurate party names |
| Thai digits in `date` | Add format instruction: "use Arabic digits" | Clean date strings |


---

## Appendix: GET endpoint alternative

The LLMObs Export API also has a GET variant with query parameters — useful for quick one-off lookups:

```bash
curl -G "https://api.us3.datadoghq.com/api/v2/llm-obs/v1/spans/events" \
  -H "DD-API-KEY: $DD_API_KEY" \
  -H "DD-APPLICATION-KEY: $DD_APP_KEY" \
  --data-urlencode "filter[span_kind]=workflow" \
  --data-urlencode "filter[ml_app]=gemini-ss6_1" \
  --data-urlencode "filter[from]=2026-02-21T09:40:00Z" \
  --data-urlencode "filter[to]=2026-02-21T11:30:00Z" \
  --data-urlencode "page[limit]=5000"
```

Both POST (search) and GET (list) return the same response schema.